In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
from tqdm import tqdm

current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
if project_root not in sys.path:
    sys.path.append(project_root)

from minisearch.search import Searcher 
from minisearch.index import PositionalIndex

import aux
import metrics

In [3]:
aux.download_and_sample(force_reload=False, n_queries=2000, n_docs=100000)

Dataset already exists. Skipping download.


In [4]:
docs_df, queries_df, qrels = aux.load_data()

index, int_to_str_id = aux.build_index(docs_df)

weight_sets = [
    ("bm25_only",        [0.0, 1.0, 0.0, 0.0]),
    ("overlap_only",     [1.0, 0.0, 0.0, 0.0]),
    ("all_1",            [1.0, 1.0, 1.0, 1.0]),
    ("bm25_cov",         [0.0, 1.0, 0.5, 0.0]),
    ("bm25_title",       [0.0, 1.0, 0.0, 0.5]),
    ("bm25_cov_title",   [0.0, 1.0, 0.5, 0.5]),
    ("bm25_cov_strong",  [0.0, 1.0, 1.0, 0.0]),
    ("bm25_title_strong",[0.0, 1.0, 0.0, 1.0]),
    ("bm25_small_bonus", [0.1, 1.0, 0.2, 0.2]),
]

results = []

for name, w in weight_sets:
    searcher = Searcher(index, w)

    mrr_scores, ndcg_scores = [], []
    for _, row in tqdm(queries_df.iterrows(), total=len(queries_df), desc=name):
        qid, text = row["query_id"], row["text"]
        if qid not in qrels:
            continue
        targets = qrels[qid]

        results_int = [doc_id for doc_id, _ in searcher.search(text)]
        results_str = [int_to_str_id[i] for i in results_int]

        mrr_scores.append(metrics.mrr(results_str, targets, k=10))
        ndcg_scores.append(metrics.ndcg(results_str, targets, k=10))

    mrr = float(np.mean(mrr_scores)) if mrr_scores else 0.0
    ndcg = float(np.mean(ndcg_scores)) if ndcg_scores else 0.0
    results.append((name, w, mrr, ndcg))

results.sort(key=lambda x: x[3], reverse=True)

print("\nTop configs by NDCG@10:")
for name, w, mrr, ndcg in results[:10]:
    print(f"{name:16s} w={w} | MRR@10={mrr:.4f} NDCG@10={ndcg:.4f}")


Indexing...


bm25_small_bonus: 100%|██████████| 2000/2000 [00:59<00:00, 33.41it/s]


Top configs by NDCG@10:
bm25_title_strong w=[0.0, 1.0, 0.0, 1.0] | MRR@10=0.4479 NDCG@10=0.4665
bm25_title       w=[0.0, 1.0, 0.0, 0.5] | MRR@10=0.4449 NDCG@10=0.4642
bm25_cov_title   w=[0.0, 1.0, 0.5, 0.5] | MRR@10=0.4446 NDCG@10=0.4640
bm25_small_bonus w=[0.1, 1.0, 0.2, 0.2] | MRR@10=0.4433 NDCG@10=0.4628
all_1            w=[1.0, 1.0, 1.0, 1.0] | MRR@10=0.4436 NDCG@10=0.4622
bm25_only        w=[0.0, 1.0, 0.0, 0.0] | MRR@10=0.4420 NDCG@10=0.4616
bm25_cov_strong  w=[0.0, 1.0, 1.0, 0.0] | MRR@10=0.4421 NDCG@10=0.4616
bm25_cov         w=[0.0, 1.0, 0.5, 0.0] | MRR@10=0.4419 NDCG@10=0.4614
overlap_only     w=[1.0, 0.0, 0.0, 0.0] | MRR@10=0.1697 NDCG@10=0.2089
